In [6]:
%load_ext autoreload
%autoreload 2
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from sklearn import preprocessing,model_selection,metrics,ensemble
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def prob_to_bi(data):
    ret=data.apply(lambda x: 0 if x<0.5 else 1)
    return ret

lr=pd.read_csv("../results/lr.csv")
rfc=pd.read_csv("../results/rfc.csv")
svm=pd.read_csv("../results/svm.csv")

metrics.recall_score(rfc['isfraud'],prob_to_bi(rfc['rfc_preds']))



0.5412151467186168

In [4]:
# dd=dd.merge(svm,on=['fold','isfraud'])


,fold,isfraud,lr_preds,rfc_preds
0,0,0,1.086876e-08,0.062593
1,0,0,1.086876e-08,0.052092
2,0,0,1.086876e-08,0.052092
3,0,0,1.086876e-08,0.052092
4,0,0,1.086876e-08,0.062593
...,...,...,...,...
333490670,4,1,1.000000e+00,0.421137
333490671,4,1,1.000000e+00,0.641541
333490672,4,1,1.000000e+00,0.562828
333490673,4,1,1.000000e+00,0.641541


In [15]:
res=lr.copy()
res['rfc_preds']=rfc['rfc_preds']
res['svm_preds']=svm['svm_preds']
res['mean']=res[['lr_preds','rfc_preds','svm_preds']].mean(axis=1)
metrics.recall_score(res['isfraud'],prob_to_bi(res['mean']))

0.8788506027030317

In [16]:
import xgboost
xgb=xgboost.XGBClassifier(verbosity=0)
pp=[]
for fold in range(5):
    train_df=res[res['fold']!=fold]
    val_df=res[res['fold']==fold]
    trainx,trainy=train_df[['lr_preds','rfc_preds','svm_preds']].values,train_df['isfraud'].values
    valx,valy=val_df[['lr_preds','rfc_preds','svm_preds']].values,val_df['isfraud'].values
    xgb.fit(trainx,trainy)
    preds=xgb.predict(valx)
    val_df.loc[:,'xgb']=preds
    pp.append(val_df[['xgb']])
dd=pd.concat(pp,axis=0)
metrics.recall_score(res['isfraud'],dd['xgb'])


/Users/home2/.conda/envs/py38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/var/folders/th/m78zdlwx5fb59574g4xwz9p40000gp/T/ipykernel_45917/2337993502.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df.loc[:,'xgb']=preds
/Users/home2/.conda/envs/py38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClass

0.8585169852672592

In [ ]:
main_data

,fold,isfraud,lr_preds,rfc_preds,svm_preds
0,0,0,1.086876e-08,0.062593,0.035011
1,0,0,1.086879e-08,0.052092,0.047847
2,0,0,1.086891e-08,0.052092,0.060408
3,0,0,1.086884e-08,0.052092,0.051205
4,0,0,1.086871e-08,0.062593,0.030466
...,...,...,...,...,...
48208,4,1,1.000000e+00,0.421137,0.999991
48209,4,1,1.000000e+00,0.641541,1.000000
48210,4,1,1.000000e+00,0.562828,0.999999
48211,4,1,1.000000e+00,0.641541,1.000000


In [4]:
import os
import pandas as pd
import numpy as np
from sklearn import metrics
import xgboost
import optuna
import lr,svm,rfc


def train_fold(main_data,fold,params):
    train_df = main_data[main_data['fold']!=fold]
    val_df = main_data[main_data['fold']==fold]
    trainx,trainy=train_df.drop(['isfraud','fold'],axis=1),train_df['isfraud']
    valx,valy=val_df.drop(['isfraud','fold'],axis=1),val_df['isfraud']

    model=xgboost.XGBClassifier(verbosity=0,**params)
    model.fit(trainx,trainy)
    preds=model.predict(valx)
    recall=metrics.accuracy_score(valy,preds)
    return recall


def objective(trial):
    
    params_lr={'penalty':trial.suggest_categorical('penalty_lr',['l1','l2']),
               'C':trial.suggest_uniform('C_lr',0,1),
               'solver':trial.suggest_categorical('solver_lr',['saga']),
               'class_weight': trial.suggest_categorical('class_weight_lr', ['balanced',None]),
    }
    
    params_rfc={
        'n_estimators':trial.suggest_int('n_estimators_rf',1,10,1,log=False),
        'criterion':trial.suggest_categorical('critrion_rf',['gini', 'entropy']),
        'max_depth':trial.suggest_categorical('max_depth_rf',[2]),
        'bootstrap':trial.suggest_categorical('bootstrap_rf',[True]),
        'max_features':trial.suggest_uniform('max_features_rf',0,0.8),
        'class_weight': trial.suggest_categorical('class_weight_rf', ['balanced','balanced_subsample',None]),            
                }
    

    params_svm={
        'C':trial.suggest_uniform('C_svm',0,5),
        'kernel':trial.suggest_categorical('kernel_svm',['linear','rbf','sigmoid']),
        'class_weight':trial.suggest_categorical('class_weight_svm',['balanced',None]),
    }
    
    params_xgb = {
        'n_estimators': trial.suggest_int('n_estimators_xg',1, 100,2,log=False),  # nmber of boosting steps
        'max_depth': trial.suggest_int('max_depth_xg',1, 20,1,log=False),  # max depth of base estimator
    # weight in combining each steap of the boost must be small if n_estimators are very large
        'learning_rate': trial.suggest_loguniform('learning_rate_xg',0.001, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda_xg',1e-9, 5),  # l2 ref param
        'max_leaves': trial.suggest_int('max_leaves_xg',1, 20,2,log=False),
        'booster': trial.suggest_categorical('booster_xg',['gbtree', 'gblinear', 'dart']),
    # regularization by only using a randaom set given fraction of rows for traning each boost step
        'subsample': trial.suggest_uniform('subsample_xg',0.1, 1.0),
        'sampling_method': trial.suggest_categorical('sampling_method_xg',['uniform']), #, 'gradient_based']),
    # regularization by only using a random set of given fraction of column in each boost step
        'colsample_bytree': trial.suggest_uniform('colsample_bytree_xg',0.1, 1.0),
    # 'predictor': Categorical(['gpu_predictor']),
        'grow_policy': trial.suggest_categorical('grow_policy_xg',['depthwise', 'lossguide'])
        }
    
    
    
    lr_preds=lr.train_full(params_lr) 
    rfc_preds=rfc.train_full(params_rfc)
    svm_preds=svm.train_full(params_svm)
    main_data=lr_preds
    main_data['rfc_preds']=rfc_preds['rfc_preds']
    main_data['svm_preds']=svm_preds['svm_preds']

    scores = []
    for fold in range(5):
        recall_score = train_fold(main_data, fold, params)
        scores.append(recall_score)
    return np.mean(scores)


def optimize(objective):
    study=optuna.create_study(directions=['maximize'])
    study.optimize(objective,n_trials=1)
    best_trial=study.best_trial
    return best_trial.values,best_trial.params

#     meta_data=best_trial.params
#     meta_data['score']=best_trial.values
#     trial_summary=study.trials_dataframe()
#     results=pd.concat(results,trial_summary)
#     pickle.dump(meta_data,open(f"meta_data.p",'wb'))
#     trial_summary.to_csv('trial_summary.csv')


score,params=optimize(objective)

[I 2022-11-23 20:58:24,438] A new study created in memory with name: no-name-6cb0cfcb-0b69-45e8-80c6-19b366697cd2
/var/folders/th/m78zdlwx5fb59574g4xwz9p40000gp/T/ipykernel_44723/330312877.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'C':trial.suggest_uniform('C_lr',0,1),
/var/folders/th/m78zdlwx5fb59574g4xwz9p40000gp/T/ipykernel_44723/330312877.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'max_features':trial.suggest_uniform('max_features_rf',0,0.8),
/var/folders/th/m78zdlwx5fb59574g4xwz9p40000gp/T/ipykernel_44723/330312877.py:42: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. Se

In [13]:
lr_params = pd.read_pickle("../model_params/lr.p")
svm_params = pd.read_pickle("../model_params/svm.p")
rfc_params=pd.read_pickle("../model_params/rfc.p")

In [2]:
import pickle
import utils
import os
import pandas as pd
import numpy as np
from sklearn import metrics
import xgboost
import optuna
from sklearn import ensemble, svm, linear_model
from config import config
# from model_params import lr, rfc, svm


lr_params = pd.read_pickle("../model_params/lr.p")
svm_params = pd.read_pickle("../model_params/svm.p")
rfc_params = pd.read_pickle("../model_params/rfc.p")

lr = linear_model.LogisticRegression(**lr_params)
svm = svm.SVC(probability=True)
rfc = ensemble.RandomForestClassifier(**rfc_params)
folds_df = pd.read_csv("../data/folds/folds.csv")


def train(models, names, pre_funcs=None):
    for model, name, funcs in zip(models, names, pre_funcs):
        folds = pd.read_csv("../data/folds/folds.csv")

        if funcs != None:
            for func in funcs:
                func(folds)

        for fold in range(config.num_folds):
            train = folds[folds['fold'] != fold].reset_index(drop=True)
            val = folds[folds['fold'] == fold]
            trainx, trainy = train.drop(
                ['fold', 'isfraud'], axis=1), train['isfraud']
            valx, valy = val.drop(['fold', 'isfraud'], axis=1), val['isfraud']
            model.fit(trainx, trainy)
            preds = model.predict_proba(valx)[:, 1]
            folds_df.loc[list(val.index.values), name] = preds
    return folds_df


/Users/home2/.conda/envs/py38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [5]:
lr_params = pd.read_pickle("../model_params/lr.p")
svm_params = pd.read_pickle("../model_params/svm.p")
rfc_params = pd.read_pickle("../model_params/rfc.p")

from sklearn.svm import SVC

lr = linear_model.LogisticRegression(**lr_params)
svm_ = SVC(probability=True)
rfc = ensemble.RandomForestClassifier(**rfc_params)
folds_df = pd.read_csv("../data/folds/folds.csv")

models=[lr,svm_,rfc]
names=['lr_preds','svm_preds','rfc_preds']
pre_funcs=[[utils.preprocess],None,None]
pred_folds=train(models,names,pre_funcs)

/Users/home2/.conda/envs/py38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/home2/.conda/envs/py38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/home2/.conda/envs/py38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/home2/.conda/envs/py38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/home2/.conda/envs/py38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [6]:
pred_folds


,step,type,amount,oldbalanceorg,newbalanceorig,oldbalancedest,newbalancedest,isfraud,isflaggedfraud,fold,lr_preds,svm_preds,rfc_preds
0,1,3.0,9839.64,170136.00,160296.36,0.00,0.00,0,0,0,0.011250,0.034866,0.025014
1,1,3.0,1864.28,21249.00,19384.72,0.00,0.00,0,0,0,0.011446,0.047662,0.035556
2,1,3.0,11668.14,41554.00,29885.86,0.00,0.00,0,0,0,0.011452,0.060191,0.035556
3,1,3.0,7817.71,53860.00,46042.29,0.00,0.00,0,0,0,0.011419,0.051012,0.017559
4,1,3.0,7107.77,183195.00,176087.23,0.00,0.00,0,0,0,0.011220,0.030336,0.025014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48208,743,1.0,339682.13,339682.13,0.00,0.00,339682.13,1,0,4,1.000000,0.999991,0.890982
48209,743,4.0,6311409.28,6311409.28,0.00,0.00,0.00,1,0,4,1.000000,1.000000,0.919302
48210,743,1.0,6311409.28,6311409.28,0.00,68488.84,6379898.11,1,0,4,1.000000,0.999999,0.890982
48211,743,4.0,850002.52,850002.52,0.00,0.00,0.00,1,0,4,1.000000,1.000000,0.919302


In [13]:
metrics.recall_score(pred_folds['isfraud'],prob_to_bi(pred_folds['svm_preds']))

0.7597710946061123

In [62]:
score = [0.9902727367001971]
params = {'n_estimators': 61,
          'max_depth': 3,
          'learning_rate': 0.018216712827287075,
          'reg_lambda': 0.001956540798030425,
          'max_leaves': 7,
          'booster': 'gbtree',
          'subsample': 0.3439059868534558,
          'sampling_method': 'uniform',
          'colsample_bytree': 0.3883668685088,
          'grow_policy': 'depthwise'}

clf=xgboost.XGBClassifier(**params)

In [75]:
vx,vy=main_data[main_data['fold']==4][['lr_preds','rfc_preds','svm_preds']],main_data[main_data['fold']==4]['isfraud']
tx,ty=main_data[main_data['fold']!=4][['lr_preds','rfc_preds','svm_preds']],main_data[main_data['fold']!=4]['isfraud']

In [76]:
clf.fit(tx,ty)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3883668685088,
              enable_categorical=False, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.018216712827287075,
              max_delta_step=0, max_depth=3, max_leaves=7, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=61, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=0.001956540798030425,
              sampling_method='uniform', scale_pos_weight=1,
              subsample=0.3439059868534558, tree_method='exact',
              validate_parameters=1, ...)

In [86]:
clf.

In [82]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr

LogisticRegression()

In [4]:
from utils import preprocess
import pandas as pd
aa=[preprocess]
for i in aa:
    folds = pd.read_csv("/Users/home2/Documents/PROJECTS/fraud_transaction_detection/data/folds/folds.csv")
    print(folds.head())
    ff=i(folds)
    print(ff.head())

   step  type    amount  ...  isfraud  isflaggedfraud  fold
0     1   3.0   9839.64  ...        0               0     0
1     1   3.0   1864.28  ...        0               0     0
2     1   3.0  11668.14  ...        0               0     0
3     1   3.0   7817.71  ...        0               0     0
4     1   3.0   7107.77  ...        0               0     0

[5 rows x 10 columns]
   step  type    amount  ...  isfraud  isflaggedfraud  fold
0     1   3.0 -0.316230  ...        0               0     0
1     1   3.0 -0.323205  ...        0               0     0
2     1   3.0 -0.314630  ...        0               0     0
3     1   3.0 -0.317998  ...        0               0     0
4     1   3.0 -0.318619  ...        0               0     0

[5 rows x 10 columns]


In [ ]:
class something:
    def __init__(self,preprocess_functions,params,model,preprocess=None):
        self.preprocess=preprocess
        if self.preprocess==True:
            self.preprocess_functions=preprocess_functions
        self.params=params
        self.model=model
        
        
    

    def train_fold(self,fold,hyper_params):
        folds = pd.read_csv("/Users/home2/Documents/PROJECTS/fraud_transaction_detection/data/folds/folds.csv")
        
        for func in self.preprocess_functions:
            fold=func(fold)
            
        train_df = folds[folds['fold'] != fold].reset_index(drop=True)
        val_df = folds[folds['fold'] == fold].reset_index(drop=True)
        trainx, trainy = train_df.drop(['isfraud','fold'], axis=1), train_df['isfraud']
        valx, valy = val_df.drop(['isfraud','fold'], axis=1), val_df['isfraud']
        
        model = self.model.set_params(**hyper_params)
        model.fit(trainx, trainy)
        preds = model.predict(valx)
        return metrics.recall_score(valy,preds)

        
    def objective(self,trial):
        recalls = []
        for fold in range(config.num_folds):
            recall = self.train_fold(fold,self.params_lr)
            recalls.append(recall)
        return np.mean(recalls)

    def optimize(objective):
        study=optuna.create_study(directions=['maximize'])
        study.optimize(objective, n_trials=config.n_trials_lr)
        best_trial=study.best_trial
        return best_trial.values,best_trial.params


    if __name__=="__main__":
        values,params=optimize(objective=objective)
        print(f"values={values},params={params}")
        pickle.dump(params,open("../model_params/svm.p"))